<a href="https://colab.research.google.com/github/tanoManzo/mimic_trajectories/blob/dev/MIMIC_TRAJECTORY_embeddings_04_03_collect_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Collection, Embeddings and Attitude

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
notes_path = "/content/drive/MyDrive/NIH/Data/"

In [3]:
file_name = "ICUSTAYS.csv"
df_ward = pd.read_csv(notes_path+file_name)
df_ward

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202
...,...,...,...,...,...,...,...,...,...,...,...,...
61527,59806,94944,143774,201233,metavision,CSRU,CSRU,15,15,2104-04-15 10:18:16,2104-04-17 14:51:00,2.1894
61528,59807,94950,123750,283653,metavision,CCU,CCU,7,7,2155-12-08 05:33:16,2155-12-10 17:24:58,2.4942
61529,59808,94953,196881,241585,metavision,SICU,SICU,57,57,2160-03-03 16:09:11,2160-03-04 14:22:33,0.9259
61530,59809,94954,118475,202802,metavision,CSRU,CSRU,15,15,2183-03-25 09:53:10,2183-03-27 17:55:03,2.3346


In [18]:
subject_id_csru = df_ward[(df_ward['FIRST_CAREUNIT']=='CSRU') & (df_ward['LAST_CAREUNIT']=='CSRU')].SUBJECT_ID.unique()
print(f'Number of Subjects in CSRU: {len(subject_id_csru)}')
hosp_stay_id_csru = df_ward[(df_ward['FIRST_CAREUNIT']=='CSRU') & (df_ward['LAST_CAREUNIT']=='CSRU')].HADM_ID.unique()
print(f'Number of Hospital stays in CSRU: {len(hosp_stay_id_csru)}')
subject_id_micu = df_ward[(df_ward['FIRST_CAREUNIT']=='MICU') & (df_ward['LAST_CAREUNIT']=='MICU')].SUBJECT_ID.unique()
print(f'Number of Subjects in MICU: {len(subject_id_micu)}')
hosp_stay_id_micu = df_ward[(df_ward['FIRST_CAREUNIT']=='MICU') & (df_ward['LAST_CAREUNIT']=='MICU')].HADM_ID.unique()
print(f'Number of Hospital stays in MICU: {len(hosp_stay_id_micu)}')

Number of Subjects in CSRU: 7944
Number of Hospital stays in CSRU: 8385
Number of Subjects in MICU: 14925
Number of Hospital stays in MICU: 19165


In [8]:
file_name = "DIAGNOSES_ICD.csv"
df_icd = pd.read_csv(notes_path+file_name)
df_icd_first = df_icd[df_icd['SEQ_NUM']==1]
df_icd_first_csru = df_icd_first[df_icd_first['SUBJECT_ID'].isin(subject_id_csru)]
print(f'Number of unique ICD 9 first sequence from CSRU subject: {df_icd_first_csru.ICD9_CODE.nunique()}')
icd_csru = df_icd_first_csru.ICD9_CODE.value_counts().nlargest(5)
print('Top 5 Popular ICD9 in the CSRU population with duplicates:')
print("code  #subjects")
icd_csru

Number of unique ICD 9 first sequence from CSRU subject: 822
Top 5 Popular ICD9 in the CSRU population with duplicates:
code  #subjects


41401    2984
4241     1051
41071     741
4240      530
4412      165
Name: ICD9_CODE, dtype: int64

In [9]:
condition = []
for item in df_icd['ICD9_CODE'].values:

  if str(item).isdigit():
    item = float(item)
    if (item>799 and item<1000):
      condition.append(True)
    else:
      condition.append(False)
  else:
      condition.append(False)

df_8xx = df_icd[condition]
df_8xx_csru = df_8xx[df_8xx['SUBJECT_ID'].isin(subject_id_csru)]
icd8xx_subjects_ids = df_8xx_csru['SUBJECT_ID'].unique()
icd8xx_hosp_ids = df_8xx_csru['HADM_ID'].unique()
print(f'Number of subjects with a first sequence ICD 9 code [800,900) : {len(icd8xx_subjects_ids)}')
print(f'Number of hospital stays with a first sequence ICD 9 code [800,900) : {len(icd8xx_hosp_ids)}')

Number of subjects with a first sequence ICD 9 code [800,900) : 170
Number of hospital stays with a first sequence ICD 9 code [800,900) : 185


In [10]:
print(df_8xx_csru['ICD9_CODE'].unique())

['00845' '920' '0931' '936' '00863' '0860' '0940' '00843' '990']


In [11]:
patients = "PATIENTS.csv"
df_patients = pd.read_csv(notes_path+patients)
df_patients_icd8xx_csru = df_patients[df_patients['SUBJECT_ID'].isin(icd8xx_subjects_ids)]
df_patients_icd8xx_csru.info()
print('Subjects Expired (1), Alive (0): ')
print(df_patients_icd8xx_csru['EXPIRE_FLAG'].value_counts())
print(df_patients_icd8xx_csru['GENDER'].value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 6 to 46449
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ROW_ID       170 non-null    int64 
 1   SUBJECT_ID   170 non-null    int64 
 2   GENDER       170 non-null    object
 3   DOB          170 non-null    object
 4   DOD          108 non-null    object
 5   DOD_HOSP     69 non-null     object
 6   DOD_SSN      85 non-null     object
 7   EXPIRE_FLAG  170 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 12.0+ KB
Subjects Expired (1), Alive (0): 
1    108
0     62
Name: EXPIRE_FLAG, dtype: int64
M    86
F    84
Name: GENDER, dtype: int64


In [12]:
df_gcs_total_subjects = df_patients_icd8xx_csru[df_patients_icd8xx_csru['SUBJECT_ID'].isin(icd8xx_subjects_ids)]
df_gcs_total_subjects['SUBJECT_ID'].nunique()

170

In [5]:
note_events = "NOTEEVENTS.csv"
df_note_events = pd.read_csv(notes_path+note_events)
df_note_events.info()

<ipython-input-5-8fd00f55f791>:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_note_events = pd.read_csv(notes_path+note_events)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083180 entries, 0 to 2083179
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ROW_ID       int64  
 1   SUBJECT_ID   int64  
 2   HADM_ID      float64
 3   CHARTDATE    object 
 4   CHARTTIME    object 
 5   STORETIME    object 
 6   CATEGORY     object 
 7   DESCRIPTION  object 
 8   CGID         float64
 9   ISERROR      float64
 10  TEXT         object 
dtypes: float64(3), int64(2), object(6)
memory usage: 174.8+ MB


In [ ]:
df_notes_total_hosp_stays_micu = df_note_events[df_note_events['HADM_ID'].isin(hosp_stay_id_micu)]
df_notes_total_hosp_stays_micu.sort_values(by=['HADM_ID','CHARTTIME'])
df_notes_total_hosp_stays_micu_social_worker = df_notes_total_hosp_stays_micu[df_notes_total_hosp_stays_micu['CATEGORY']=='Social Work']
#df_notes_total_hosp_stays_micu.info()
print(f'Number of Hospital Stays with Social Worker notes in MICU: {len(df_notes_total_hosp_stays_micu_social_worker.HADM_ID.unique())}')
df_notes_total_hosp_stays_csru = df_note_events[df_note_events['HADM_ID'].isin(hosp_stay_id_csru)]
df_notes_total_hosp_stays_csru.sort_values(by=['HADM_ID','CHARTTIME'])
df_notes_total_hosp_stays_csru_social_worker = df_notes_total_hosp_stays_csru[df_notes_total_hosp_stays_csru['CATEGORY']=='Social Work']
#df_notes_total_hosp_stays.info()
print(f'Number of Hospital Stays with Social Worker notes in CSRU: {len(df_notes_total_hosp_stays_csru_social_worker.HADM_ID.unique())}')

Subjects with social worker notes

In [37]:
df_notes_total_subjects_csru = df_note_events[df_note_events['SUBJECT_ID'].isin(subject_id_csru)]
df_notes_total_subjects_csru_social_worker = df_notes_total_subjects_csru[df_notes_total_subjects_csru['CATEGORY']=='Social Work']
print(f'Number of Subjects with Social Worker notes in MICU: {len(df_notes_total_subjects_csru_social_worker.SUBJECT_ID.unique())}')

Number of Subjects with Social Worker notes in MICU: 107


In [38]:
df_notes_total_subjects_micu = df_note_events[df_note_events['SUBJECT_ID'].isin(subject_id_micu)]
df_notes_total_subjects_micu_social_worker = df_notes_total_subjects_micu[df_notes_total_subjects_micu['CATEGORY']=='Social Work']
print(f'Number of Subjects with Social Worker notes in MICU: {len(df_notes_total_subjects_micu_social_worker.SUBJECT_ID.unique())}')

Number of Subjects with Social Worker notes in MICU: 707


In [28]:
print(df_note_events['CATEGORY'].unique())

['Discharge summary' 'Echo' 'ECG' 'Nursing' 'Physician ' 'Rehab Services'
 'Case Management ' 'Respiratory ' 'Nutrition' 'General' 'Social Work'
 'Pharmacy' 'Consult' 'Radiology' 'Nursing/other']


In [29]:
import re
df_notes_icd8xx_csru_social = df_notes_total_hosp_stays_csru.merge(df_patients[['SUBJECT_ID', 'EXPIRE_FLAG']], on=['SUBJECT_ID'], how='inner')
#df_notes_icd8xx_csru_social = df_notes_icd8xx_csru_social.merge(df_8xx_csru[['SUBJECT_ID', 'ICD9_CODE','HADM_ID']], on=['SUBJECT_ID','HADM_ID'], how='inner')
df_notes_icd8xx_csru_social = df_notes_icd8xx_csru_social[(df_notes_icd8xx_csru_social['CATEGORY']=='Nursing') | (df_notes_icd8xx_csru_social['CATEGORY']=='Nursing/other') | (df_notes_icd8xx_csru_social['CATEGORY']=='Social Work')]

# regex pattern
pattern = r'(?i)(social work note:|social work note-|social work note\n\n|social work note\n|social work:|social work-|social work\n\n|social work\n|social:|soc:|soc;|soc-|social-|social -|social \n|social\n\n|social\n)(.*?)(?:\n|$)'

# grabs multiple results if the pattern shows more than once
df_notes_icd8xx_csru_social['SOCIAL'] = df_notes_icd8xx_csru_social['TEXT'].apply(lambda x: ' '.join([match[1] for match in re.findall(pattern, x.lower(), flags=re.DOTALL)]) if re.findall(pattern, x.lower(), flags=re.DOTALL) else '')

#df_notes_icd8xx_tsicu_social = df_notes_icd8xx_tsicu_social[df_notes_icd8xx_tsicu_social['TEXT'].str.lower().str.contains('social work', regex=True)] # grabs only "social work" examples
#df_notes_icd8xx_tsicu_social['SOCIAL'] = df_notes_icd8xx_tsicu_social['TEXT'].str.lower().str.extract(pattern, flags=re.DOTALL)[1]  # grabs only the first result of regex pattern

#df_notes_icd8xx_tsicu_social = df_notes_icd8xx_tsicu_social.sort_values(by=['HADM_ID','CHARTTIME']) # for testing
#df_notes_icd8xx_tsicu_social = df_notes_icd8xx_tsicu_social[df_notes_icd8xx_tsicu_social['EXPIRE_FLAG'] == 1] # for testing
#df_notes_icd8xx_tsicu_social = df_notes_icd8xx_tsicu_social[df_notes_icd8xx_tsicu_social['HADM_ID'] == 108474] # for testing

df_notes_icd8xx_csru_social = df_notes_icd8xx_csru_social.sort_values(by='SOCIAL', key=lambda x: x.str.len(), ascending=False)
df_notes_icd8xx_csru_social.info()
df_notes_icd8xx_csru_social.head(300)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92154 entries, 36824 to 199898
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ROW_ID       92154 non-null  int64  
 1   SUBJECT_ID   92154 non-null  int64  
 2   HADM_ID      92154 non-null  float64
 3   CHARTDATE    92154 non-null  object 
 4   CHARTTIME    92154 non-null  object 
 5   STORETIME    92154 non-null  object 
 6   CATEGORY     92154 non-null  object 
 7   DESCRIPTION  92154 non-null  object 
 8   CGID         92154 non-null  float64
 9   ISERROR      45 non-null     float64
 10  TEXT         92154 non-null  object 
 11  EXPIRE_FLAG  92154 non-null  int64  
 12  SOCIAL       92154 non-null  object 
dtypes: float64(3), int64(3), object(7)
memory usage: 9.8+ MB


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,EXPIRE_FLAG,SOCIAL
36824,1485985,17081,103931.0,2138-08-13,2138-08-13 15:52:00,2138-08-13 15:59:00,Nursing/other,Report,17765.0,NaN,SOCIAL WORK NOTE: Met this afternoon for one ...,1,met this afternoon for one hour with pt's wi...
26057,1519601,19558,101168.0,2172-09-30,2172-09-30 14:53:00,2172-09-30 15:02:00,Nursing/other,Report,17765.0,NaN,SOCIAL WORK NOTE: New trauma pt on SICU admit...,0,new trauma pt on sicu admitted [**9-29**] s/...
125737,1527284,20155,152446.0,2143-07-28,2143-07-28 05:24:00,2143-07-28 06:02:00,Nursing/other,Report,17512.0,NaN,"[** 1176**] NPN\nSEE 2 NOTES FROM THIS SHIFT, ...",1,son [**name (ni) **] present when bleeding be...
20713,1445244,14019,188498.0,2167-08-11,2167-08-11 15:25:00,2167-08-11 15:41:00,Nursing/other,Report,17765.0,NaN,SOCIAL WORK NOTE: New trauma patient on CTIC/...,1,new trauma patient on ctic/sicu. pt is an u...
48022,1450507,14443,185831.0,2124-09-11,2124-09-11 13:46:00,2124-09-11 13:55:00,Nursing/other,Report,17765.0,NaN,"SOCIAL WORK NOTE: Met with pt's wife, [**Name...",0,"met with pt's wife, [**name (ni) 1257**], to..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27035,1308592,3597,162212.0,2128-04-17,2128-04-17 18:44:00,2128-04-17 18:56:00,Nursing/other,Report,16140.0,NaN,SICU NPN CONT...\nGU: urine sent off this AM f...,0,[**name (ni) 25**] wife [**doctor first name ...
50620,1515459,19206,164924.0,2120-10-01,2120-10-01 12:27:00,2120-10-01 12:29:00,Nursing/other,Report,17765.0,NaN,"SOCIAL WORK NOTE: Pt's partner, [**Name (NI) ...",0,"pt's partner, [**name (ni) **], mother, [**n..."
60972,1510239,18874,101982.0,2195-10-06,2195-10-06 16:37:00,2195-10-06 16:39:00,Nursing/other,Report,17765.0,NaN,SOCIAL WORK NOTE: Please see ED note from thi...,1,please see ed note from this sw earlier in a...
22238,1272116,803,143339.0,2111-10-17,2111-10-17 04:58:00,2111-10-17 05:50:00,Nursing/other,Report,15255.0,NaN,S/P FALL SDH C/SICU NSG ADMIT NOTE.\nPT ARRIVE...,1,pt lives w/ daughter on [**initials (namepatte...


In [ ]:
import re

df_notes_micu_social = df_notes_total_hosp_stays_micu.merge(df_patients[['SUBJECT_ID', 'EXPIRE_FLAG']], on=['SUBJECT_ID'], how='inner')
#df_notes_icd8xx_csru_social = df_notes_icd8xx_csru_social.merge(df_8xx_csru[['SUBJECT_ID', 'ICD9_CODE','HADM_ID']], on=['SUBJECT_ID','HADM_ID'], how='inner')
df_notes_micu_social = df_notes_micu_social[df_notes_micu_social['CATEGORY']=='Social Work']

# regex pattern
pattern = r'(?i)(social work note:|social work note-|social work note\n\n|social work note\n|social work:|social work-|social work\n\n|social work\n|social:|soc:|soc;|soc-|social-|social -|social \n|social\n\n|social\n)(.*?)(?:\n|$)'

# grabs multiple results if the pattern shows more than once
df_notes_micu_social['SOCIAL'] = df_notes_micu_social['TEXT'].apply(lambda x: ' '.join([match[1] for match in re.findall(pattern, x.lower(), flags=re.DOTALL)]) if re.findall(pattern, x.lower(), flags=re.DOTALL) else '')

#df_notes_icd8xx_tsicu_social = df_notes_icd8xx_tsicu_social[df_notes_icd8xx_tsicu_social['TEXT'].str.lower().str.contains('social work', regex=True)] # grabs only "social work" examples
#df_notes_icd8xx_tsicu_social['SOCIAL'] = df_notes_icd8xx_tsicu_social['TEXT'].str.lower().str.extract(pattern, flags=re.DOTALL)[1]  # grabs only the first result of regex pattern

#df_notes_icd8xx_tsicu_social = df_notes_icd8xx_tsicu_social.sort_values(by=['HADM_ID','CHARTTIME']) # for testing
#df_notes_icd8xx_tsicu_social = df_notes_icd8xx_tsicu_social[df_notes_icd8xx_tsicu_social['EXPIRE_FLAG'] == 1] # for testing
#df_notes_icd8xx_tsicu_social = df_notes_icd8xx_tsicu_social[df_notes_icd8xx_tsicu_social['HADM_ID'] == 108474] # for testing

df_notes_micu_social = df_notes_micu_social.sort_values(by='SOCIAL', key=lambda x: x.str.len(), ascending=False)
df_notes_micu_social.info()
df_notes_micu_social.head(300)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1352 entries, 404803 to 348365
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ROW_ID       1352 non-null   int64  
 1   SUBJECT_ID   1352 non-null   int64  
 2   HADM_ID      1352 non-null   float64
 3   CHARTDATE    1352 non-null   object 
 4   CHARTTIME    1352 non-null   object 
 5   STORETIME    1352 non-null   object 
 6   CATEGORY     1352 non-null   object 
 7   DESCRIPTION  1352 non-null   object 
 8   CGID         1352 non-null   float64
 9   ISERROR      3 non-null      float64
 10  TEXT         1352 non-null   object 
 11  EXPIRE_FLAG  1352 non-null   int64  
 12  SOCIAL       1352 non-null   object 
dtypes: float64(3), int64(3), object(7)
memory usage: 147.9+ KB


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,EXPIRE_FLAG,SOCIAL
404803,531119,62488,191090.0,2112-03-11,2112-03-09 05:15:00,2112-03-11 09:54:04,Social Work,Social Work Progress Note,16285.0,NaN,Note: This note is from [**2112-3-9**].\nSocia...,1,[**first name9 (namepattern2) 14366**] [**doct...
557223,666648,70516,199605.0,2189-03-27,2189-03-27 19:35:00,2189-03-27 19:35:17,Social Work,Social Work Progress Note,21359.0,NaN,SOCIAL WORK: [**First Name8 (NamePattern2) **...,1,[**first name8 (namepattern2) **] [**last na...
557232,666649,70516,199605.0,2189-03-27,2189-03-27 19:35:00,2189-03-27 19:54:34,Social Work,Social Work Progress Note,21359.0,NaN,SOCIAL WORK: [**First Name8 (NamePattern2) **...,1,[**first name8 (namepattern2) **] [**last na...
47090,384162,5242,195261.0,2123-07-30,2123-07-30 15:08:00,2123-07-30 15:08:20,Social Work,Social Work Progress Note,16285.0,NaN,Social Work\n Pt. known to this writer from ...,1,pt. known to this writer from 11r. please r...
44016,582207,65901,113793.0,2139-05-14,2139-05-14 14:55:00,2139-05-14 15:23:39,Social Work,Social Work Admission Note,16399.0,NaN,Family Information\n Next of [**Doctor First...,0,"worker for support, but she did not want to..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380290,542937,43257,135365.0,2158-12-04,2158-12-04 14:30:00,2158-12-04 14:30:41,Social Work,Social Work Admission Note,16399.0,NaN,Family Information\n Next of [**Doctor First...,0,
380894,650758,76418,152251.0,2149-12-31,2149-12-31 11:11:00,2149-12-31 11:11:48,Social Work,Social Work Progress Note,19926.0,NaN,"Social Progress Note, Transplant Service: Cont...",0,
380604,436927,64292,123357.0,2141-02-13,2141-02-13 15:10:00,2141-02-13 15:10:39,Social Work,Social Work Admission Note,14081.0,NaN,Family Information\n Next of [**Doctor First...,1,
380489,539239,91586,165668.0,2107-11-04,2107-11-03 15:28:00,2107-11-04 15:28:31,Social Work,Social Work Admission Note,16399.0,NaN,Family Information\n Next of [**Doctor First...,1,


In [ ]:
PATH_TO_SAVE = "/content/drive/MyDrive/NIH/Data/"
name_file_to_save = 'NOTES_NURSING_TRAJECTORIES_CSRU_SOCIAL_ONLY_V2.csv'
df_notes_icd8xx_csru_social['SOCIAL'].to_csv(PATH_TO_SAVE+name_file_to_save,index=False)

In [ ]:
PATH_TO_SAVE = "/content/drive/MyDrive/NIH/Data/"
name_file_to_save = 'NOTES_NURSING_TRAJECTORIES_MICU_SOCIAL_WORKER_ONLY.csv'
df_notes_micu_social['TEXT'].to_csv(PATH_TO_SAVE+name_file_to_save,index=False)

In [ ]:
PATH_TO_SAVE = "/content/drive/MyDrive/NIH/Data/"
name_file_to_save = 'NOTES_NURSING_TRAJECTORIES_CSRU_TEXT_ONLY.csv'
#df_notes_icd8xx_tsicu_social['TEXT'].to_csv(PATH_TO_SAVE+name_file_to_save,index=False)

In [ ]:
PATH_TO_SAVE = "/content/drive/MyDrive/NIH/Data/"
name_file_to_save = 'NOTES_NURSING_TRAJECTORIES_CSRU_GCS_HADM.csv'
#df_notes_icd8xx_tsicu_gcs_scores.to_csv(PATH_TO_SAVE+name_file_to_save,index=False)

In [ ]:
df_note_icd8xx_tsicu = df_note_events[df_note_events['SUBJECT_ID'].isin(icd8xx_subjects_ids)]
print(f'Number of total notes: {len(df_note_icd8xx_tsicu)}')
df_nursing_progress_note_icd8xx_tsicu = df_note_icd8xx_tsicu[df_note_icd8xx_tsicu['DESCRIPTION']=='Nursing Progress Note']
print(f'Number of Nursing Progress Notes: {len(df_nursing_progress_note_icd8xx_tsicu)}')
df_discharge_note_icd8xx_tsicu = df_note_icd8xx_tsicu[(df_note_icd8xx_tsicu['CATEGORY']=='Discharge summary') ]
print(f'Number of Discharge summary: {len(df_discharge_note_icd8xx_tsicu)}')

In [ ]:
cgid_icd8xx_tsicu = df_nursing_progress_note_icd8xx_tsicu['CGID'].unique()
#print(f'Number of caregivers: {len(cgid_maxicu_tsicu)}')

In [ ]:
caregivers = "CAREGIVERS.csv"
df_caregivers = pd.read_csv(notes_path+caregivers)
df_caregivers[df_caregivers['CGID'].isin(cgid_icd8xx_tsicu)].LABEL.value_counts()

In [ ]:
name_file= "ADMISSIONS.csv"
df_adm = pd.read_csv(notes_path+name_file)
df_adm_icd8xx_tsicu = df_adm[df_adm['SUBJECT_ID'].isin(cgid_icd8xx_tsicu)]
df_adm_icd8xx_tsicu

Report

In [ ]:
print(f'Number of Subjects in TSICU: {len(subject_id_tsicu)}')
print(f'Number of subjects with a first sequence ICD 9 code [800,900) : {len(icd8xx_subjects_ids)}')
print('')
print('Subjects Expired (1), Alive (0): ')
print(df_patients_icd8xx_tsicu['EXPIRE_FLAG'].value_counts())
print('')
print('Gender:')
print(df_patients_icd8xx_tsicu['GENDER'].value_counts())
print('')
print(f'Number of total notes: {len(df_note_icd8xx_tsicu)}')
print(f'Number of Nursing Progress Notes: {len(df_nursing_progress_note_icd8xx_tsicu)}')
print(f'Number of Discharge summary: {len(df_discharge_note_icd8xx_tsicu)}')
print(f'Number of caregivers: {len(cgid_icd8xx_tsicu)}')

# END


## **Data Preprocessing**

In [ ]:
df_note_events.info()

Grab tsicu notes, merge with ward and patients tables

In [ ]:
# get new type of notes (TSICU)
df_notes_TSICU = pd.DataFrame()
#df_notes_TSICU = df_notes.merge(df_note_events[['ROW_ID', 'CATEGORY']], on='ROW_ID')
df_notes_TSICU = df_note_events.merge(df_caregivers.drop('ROW_ID', axis=1), on='CGID')
#df_notes_TSICU = df_notes_TSICU.merge(df_ward.drop('ROW_ID', axis=1), on='SUBJECT_ID')
df_notes_TSICU = df_notes_TSICU.merge(df_patients.drop('ROW_ID', axis=1), on='SUBJECT_ID')
#df_notes_TSICU = df_notes_TSICU[(df_notes_TSICU['first_careunit'] == 'TSICU') & (df_notes_TSICU['last_careunit'] == 'TSICU')].drop_duplicates()
df_notes_TSICU.info()

In [ ]:
df_notes_TSICU.head(5)

Join with ICD

In [ ]:
icd = "DIAGNOSES_ICD.csv"
df_icd = pd.read_csv(notes_path+icd)
df_icd.columns = df_icd.columns.str.upper()
df_icd = df_icd.dropna(subset=['SEQ_NUM'])
df_icd['SEQ_NUM'] = df_icd['SEQ_NUM'].astype(int)
unique_values = df_icd['SEQ_NUM'].unique()
print("unique_values: ")
print(unique_values)
idx = df_icd.groupby(['HADM_ID', 'SUBJECT_ID'])['SEQ_NUM'].idxmin()
df_icd = df_icd.loc[idx]
df_icd.info()

In [ ]:
df_notes_TSICU = df_notes_TSICU.merge(df_icd[['HADM_ID','ICD9_CODE','SEQ_NUM', 'SUBJECT_ID']], on=['HADM_ID', 'SUBJECT_ID'])
df_notes_TSICU.info()

In [ ]:
df_notes_TSICU.head(20)

In [ ]:
grouping = df_notes_TSICU.groupby(['CGID','SUBJECT_ID']).size()
df_notes_TSICU['num_of_notes'] = df_notes_TSICU.set_index(['CGID','SUBJECT_ID']).index.map(grouping)
df_notes_TSICU_filtered = df_notes_TSICU[df_notes_TSICU['num_of_notes'] > 1]
df_notes_TSICU_filtered.info()

In [ ]:
df_notes_TSICU_filtered.head(10)

Filter by number of notes

In [ ]:
gb_notes = df_notes_TSICU_filtered.groupby(['ICD9_CODE'])
num_notes = gb_notes['TEXT'].count()

print(f'number of notes: min={num_notes.min()}, max={num_notes.max()}, mean={num_notes.mean()},  median={num_notes.median()}')
sns.boxplot(x=num_notes)
sns.displot(num_notes[((num_notes>2) & (num_notes<100))])

In [ ]:
df_notes_TSICU_filtered.info()

In [ ]:
icd9_grouping = df_notes_TSICU_filtered.groupby(['ICD9_CODE']).size()
df_notes_TSICU_filtered['num_of_conditions'] = df_notes_TSICU_filtered.set_index(['ICD9_CODE']).index.map(icd9_grouping)
#df_notes_TSICU_filtered = df_notes_TSICU_filtered[(df_notes_TSICU_filtered['num_of_conditions'] >= 15) & (df_notes_TSICU_filtered['num_of_conditions'] <= 1000)]
#df_notes_TSICU_filtered = df_notes_TSICU_filtered[(df_notes_TSICU_filtered['num_of_conditions'] >= 15) & (df_notes_TSICU_filtered['num_of_conditions'] <= 100)]
df_notes_TSICU_filtered.info()
print(icd9_grouping)

In [ ]:
df_notes_TSICU_dropped_cols = df_notes_TSICU_filtered.drop(['STORETIME','CHARTDATE','CHARTTIME','DOB','DOD','DOD_HOSP','DOD_SSN'], axis=1)
df_notes_TSICU_dropped_cols.columns

select emotional words then filter rows that dont have a minimum number of them

In [ ]:
def filter_for_emotional_words(df):
  selected_words = ['pain', 'family', 'stable', 'care', 'well', 'social', 'support', 'able', 'decreased', 'warm', 'unable', 'strong', 'intact', 'good', 'times', 'aware', 'eyes', 'tolerated', 'denies', 'tolerating', 'please', 'palpable']
  print("selected_words list length: ", len(selected_words))
  pattern = '|'.join(selected_words)
  new_df = df[df['TEXT'].str.contains(pattern, regex=True)].copy()
  #filtered_data = df_notes_TSICU[df_notes_TSICU['TEXT'].str.contains(pattern, regex=True)].copy()
  new_df['num_search_words'] = new_df['TEXT'].str.count(pattern)
  #new_df = new_df[new_df['num_search_words'] >= 0]
  new_df.info()
  return new_df

In [ ]:
#filtered_data = filter_for_emotional_words(df_notes_TSICU_dropped_cols)
filtered_data = df_notes_TSICU_dropped_cols

Random selection from sample set

In [ ]:
#random_filtered_data = filtered_data.sample(n=50000, random_state=42)
random_filtered_data = filtered_data

Remove similar sentences

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def remove_similar_notes(df):
  sentences = df['TEXT'].tolist()
  vectorizer = TfidfVectorizer()
  tfidf_matrix = vectorizer.fit_transform(sentences)
  similarity_threshold = 0.8

  similar_indices = []
  for i, sentence in enumerate(sentences):  # needs to be max 1000 entries
    if any(cosine_similarity(tfidf_matrix[i], tfidf_matrix[j])[0][0] > similarity_threshold for j in range(i)):  # very slow ~10 minutes for 1000 entries, exponential complexity (more than 5 hours if 6000 entries)
      similar_indices.append(i)
  index_labels = df.index[similar_indices]
  df.drop(index_labels, inplace=True)
  return df

Inverse Document Frequency

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df_notes_icd8xx_tsicu_gcs_scores['TEXT'])

feature_names = vectorizer.get_feature_names_out()
tfidf_scores = tfidf_matrix.sum(axis=0).A1
word_score_dict = dict(zip(feature_names, tfidf_scores))

sorted_words = sorted(word_score_dict.items(), key=lambda x: x[1], reverse=True)

top_words = sorted_words[:50]

In [ ]:
print("Rank\tWord\t\t\tCount")
for rank, (word, score) in enumerate(top_words, start=1):
  print(f"{rank}\t{word.ljust(20)}\t{score}")

In [ ]:
#random_filtered_data = remove_similar_notes(random_filtered_data)
random_filtered_data.info()

Find matching conditions for both expired flags

In [ ]:
def split_set_half_expired(df):
  df['EXPIRE_FLAG'] = df['EXPIRE_FLAG'].astype(int)
  group_conditions = df.groupby(['ICD9_CODE', 'EXPIRE_FLAG'])
  def balance_conditions(g1,g2,size):
    size = min(len(g1), len(g2), size)
    return g1.sample(n=size, random_state=1), g2.sample(n=size, random_state=1)

  rows = []
  size = 50000

  for drg_desc, group in group_conditions:
    icd9, expire = drg_desc
    if (icd9, 1-expire) in group_conditions.groups:
      paired_group = group_conditions.get_group((icd9, 1-expire))
      group_sample1, group_sample2 = balance_conditions(group, paired_group, size)
      rows.append(group_sample1)
      rows.append(group_sample2)
      size -= len(group_sample1)
    if size <= 0:
      break

  new_df = pd.concat(rows)
  new_df = new_df.drop_duplicates()

  new_df.info()
  #print(new_df[['ICD9_CODE', 'EXPIRE_FLAG']])
  return new_df

  #valid_groups = [key for key, group in group_conditions.groups.keys() if (key[0], key[1], 1 - key[2]) in group_conditions.groups]
  #for key in valid_groups:
    #pass

In [ ]:
#balanced_data = split_set_half_expired(random_filtered_data)
balanced_data = random_filtered_data

Display notes

In [ ]:
pd.set_option('display.max_colwidth', 250)
pd.set_option('display.max_rows', None)
#print(balanced_data['TEXT'].head(200))
pd.reset_option('display.max_colwidth')
pd.reset_option('display.max_rows')

In [ ]:
print(balanced_data.columns)
print('\n'+"Number of rows: "+str(len(balanced_data)))

In [ ]:
balanced_data.info()

In [ ]:
no_duplicates = balanced_data.sort_values('ROW_ID').drop_duplicates(subset=['CGID', 'SUBJECT_ID'])
no_duplicates = no_duplicates.drop(['TEXT'], axis=1)  # removes text notes
no_duplicates.info()

Save to CSV

In [ ]:
# path and name of the cvs file
PATH_TO_SAVE = "/content/drive/MyDrive/NIH/Data/"
name_file_to_save = 'NOTES_NURSING_TRAJECTORIES_TSICU_COUNTS.csv'
#no_duplicates.to_csv(PATH_TO_SAVE+name_file_to_save,index=False)

Gather- top ICD9, # of: patients, deceased, CGs, notes

In [ ]:
icd9_max_val = balanced_data['num_of_conditions'].max()
icd9_max_df = balanced_data[balanced_data['num_of_conditions'] == icd9_max_val]

print("Max ICD9 code: " + icd9_max_df['ICD9_CODE'].unique())
print("Number of patients: " + str(icd9_max_df['SUBJECT_ID'].nunique()))
print("Number of deceased patients: " + str((icd9_max_df.drop_duplicates(subset='SUBJECT_ID')['EXPIRE_FLAG'] == 1).sum()))
print("Number of caregivers: " + str(icd9_max_df['CGID'].nunique()))
unique_cgid = icd9_max_df['CGID'].unique()
unique_cgid_df = icd9_max_df[icd9_max_df['CGID'].isin(unique_cgid)].drop_duplicates(subset=['CGID', 'LABEL'])
unique_cgid_df['LABEL'] = unique_cgid_df['LABEL'].fillna('UNKNOWN')
unique_cgid_df['LABEL'] = unique_cgid_df['LABEL'].str.upper()
print("Caregiver labels: " + '\n' + str(unique_cgid_df['LABEL'].value_counts()))
print("Number of caregivers: " + str(icd9_max_df['CGID'].nunique()))
print("Number of notes: " + str(len(icd9_max_df)))

deceased patients for unfiltered set

In [ ]:
len(df_notes_TSICU['SUBJECT_ID'].unique())
deceased_TSICU = df_notes_TSICU[df_notes_TSICU['EXPIRE_FLAG'] == 1]
deceased_TSICU.info()

In [ ]:
sns.set()

gb_notes = deceased_TSICU.groupby(['CGID','SUBJECT_ID'])
num_notes = gb_notes['TEXT'].count()
print(num_notes)
print(f'number of notes: min={num_notes.min()}, max={num_notes.max()}, mean={num_notes.mean()},  median={num_notes.median()}')
sns.boxplot(x=num_notes)
sns.displot(num_notes[((num_notes>2) & (num_notes<15))])

Seperate deceased and living

In [ ]:
deceased_patients = balanced_data[balanced_data['EXPIRE_FLAG'] == 1]
deceased_patients.info()

In [ ]:
living_patients = balanced_data[balanced_data['EXPIRE_FLAG'] == 0]
living_patients.info()

Tokenized words

In [ ]:
#text = df_notes_TSICU['TEXT'].str.cat(sep=' ')
#words = nltk.word_tokenize(text)  # very slow

Count words in notes

In [ ]:
#stopwords = nltk.corpus.stopwords.words('english')
#words = [word.lower() for word in words if word.lower() not in stopwords and re.match(r'\b\w+\b', word)]  # also slow but only kinda slow
#word_counts = Counter(words)
#most_common_words = word_counts.most_common(300)

In [ ]:
#print("Rank\tWord\t\t\tCount")
#for rank, word_count in enumerate(most_common_words, start=1):
  #word, count = word_count
  #print(f"{rank}\t{word.ljust(20)}\t{count}")

Emotional words

In [ ]:
#emotional_words = []
#for word_count in most_common_words:
  #word = word_count[0]
  #synsets = wn.synsets(word)
  #if synsets:
    #senti_synset = swn.senti_synset(synsets[0].name())
    #if senti_synset.pos_score() > 0.45 or senti_synset.neg_score() > 0.45:
      #emotional_words.append(word)

#emotional_words

In [ ]:
#from textblob import TextBlob
#emotional_words = []
#for word_count in most_common_words:
  #word = word_count[0]
  #tb = TextBlob(word)
  #sentiment = tb.sentiment.polarity
  #if abs(sentiment) > 0.35:
    #emotional_words.append(word)

#emotional_words